In [1]:
import pandas as pd

In [21]:
teamrank_df = pd.read_csv('kbo_teamrank_2001_2023.csv')
hitter_df = pd.read_csv('kbo_hitter_2001_2023.csv')
pitcher_df = pd.read_csv('kbo_pitcher_2001_2023.csv')

hitter_df = hitter_df.drop(columns=['순위']).rename(columns={'R': '득점'})
pitcher_df = pitcher_df.drop(columns=['순위']).rename(columns={'R': '실점'})

merged_df = pd.merge(teamrank_df, hitter_df, on=['Year', '팀명'], how='outer')
merged_df = pd.merge(merged_df, pitcher_df, on=['Year', '팀명'], how='outer')

merged_df_columns = ['Year', '순위', '팀명', '경기', '승', '패', '무', '승률', '게임차', '득점', '실점']
merged_df = merged_df[merged_df_columns]

merged_df.to_csv('sampledata.csv', index=False)

In [23]:
df = pd.read_csv('sampledata.csv')

def calculate_pythagorean_win_rate(row, n=1.83):
    try:
        runs_scored = float(row['득점'])
        runs_allowed = float(row['실점'])
        win_rate = (runs_scored ** n) / ((runs_scored ** n) + (runs_allowed ** n))
        return win_rate
    except ZeroDivisionError:
        return None
    except ValueError:
        return None

df['피타고리안승률'] = df.apply(calculate_pythagorean_win_rate, axis=1).round(4)
df['승률차'] = (df['승률'] - df['피타고리안승률']).round(4)
df['기대순위'] = df.groupby('Year')['피타고리안승률'].rank(ascending=False, method='min').astype(int)
df['기대순위-실제순위'] = df['기대순위'] - df['순위'].astype(int)

df.to_csv('pythagorean_wins.csv', index=False)